In [1]:
import random
import glob
import json
import numpy as np

## Fetch the data

In [2]:
def get_card_as_one_hot(card):
    s = ['clubs','hearts','spades','diamonds'].index(card['suit'])
    v = ([ str(i) for i in range(2,11) ] + ['J','K','Q','A']).index(card['rank'])
    return v,s

def get_cards_as_one_hot(cards):
    one_hot = np.zeros((13,4))
    for c in cards:
        one_hot[get_card_as_one_hot(c)] = 1
    return one_hot

def get_all_cards_for_hand_for_player(hand, player_index):
    return np.array([
        get_cards_as_one_hot(hand['hole_cards'][player_index]), 
        get_cards_as_one_hot(hand['community_cards'][:3]),
        get_cards_as_one_hot(hand['community_cards'][3:4]),
        get_cards_as_one_hot(hand['community_cards'][4:5])
    ])

def rotate(a, n):
    return a[n:] + a[:n]

def normalize(bets, active_player, blind):
    result = rotate(bets,active_player)
    return list(map(lambda x: x / blind, result))

def get_bets(hand, active_player):
    bets=[]
    number_of_players = len(hand['bets'][0][0])
    previous_bet = 0
    bet_index = 0
    for r in hand['bets']:
        bets.append([])
        for b in r:
            if b['on_turn'] == active_player:
                if bet_index > 1: 
                    current_bet = b['bets'][active_player]- previous_bet 
                    normalized_totals = normalize(b['bets'], active_player, hand['blind'])
                    normalized_current_bet = (current_bet)/hand['blind']
                    normalized_totals[0] -= normalized_current_bet
                    
                    bets[-1].append({ 
                        'bet': normalized_current_bet,
                        'totals': normalized_totals
                    })
            previous_bet = b['bets'][active_player]
            bet_index += 1
    return bets


def zero_last_layers(layers, round_index):
    for i in range(round_index + 1, len(layers)):
        layers[i] = np.zeros(layers[i].shape)
    return layers

def one_hot(i, length):
    one_hot = np.zeros((length,))
    one_hot[i] = 1
    return one_hot

def convert_hand(hand):
    inputs = []
    outputs = []
    for round_index, round_bets in enumerate(get_bets(hand, hand['game_winner'])):
        for this_bet in round_bets:
            nof_players = len(this_bet['totals'])
            normalized_dealer = (hand['dealer'] - hand['game_winner'] + nof_players) % nof_players
            cards = get_all_cards_for_hand_for_player(hand, hand['game_winner'])
            initial_stacks = normalize(hand['initial_stacks'], hand['game_winner'], hand['blind'])
            remaining_stack = (np.array(initial_stacks) - np.array(this_bet['totals']))
            dealer_onehot = one_hot(normalized_dealer, nof_players)
            inputs.append([
                zero_last_layers(cards, round_index), # cards
                np.array([remaining_stack, this_bet['totals'],dealer_onehot]).reshape((3, nof_players, 1))
            ])
            outputs.append([this_bet['bet']])
    return inputs, outputs


In [3]:
file = glob.glob("/data/poker/rawdata/*.json")[3]
f = open(file, 'r')
hands = json.loads(f.read())
test_hand = hands[2]

print(test_hand)
convert_hand(test_hand)

{'game_id': '587a5711cfa7010004000619', 'winner': 4, 'bets': [[{'on_turn': 3, 'type': 'raise', 'bets': [0, 0, 0, 2, 0], 'bet': 2}, {'on_turn': 4, 'type': 'raise', 'bets': [0, 0, 0, 2, 4], 'bet': 4}, {'on_turn': 0, 'type': 'fold', 'bets': [0, 0, 0, 2, 4], 'bet': 0}, {'on_turn': 1, 'type': 'fold', 'bets': [0, 0, 0, 2, 4], 'bet': 0}, {'on_turn': 2, 'type': 'allin', 'bets': [0, 0, 22, 2, 4], 'bet': 22}, {'on_turn': 3, 'type': 'fold', 'bets': [0, 0, 22, 2, 4], 'bet': 0}, {'on_turn': 4, 'type': 'call', 'bets': [0, 0, 22, 2, 22], 'bet': 18}], [], [], []], 'dealer': 2, 'hole_cards': [[{'rank': '9', 'suit': 'hearts'}, {'rank': '5', 'suit': 'clubs'}], [{'rank': '9', 'suit': 'clubs'}, {'rank': '8', 'suit': 'spades'}], [{'rank': 'A', 'suit': 'hearts'}, {'rank': '3', 'suit': 'clubs'}], [{'rank': 'K', 'suit': 'hearts'}, {'rank': '7', 'suit': 'clubs'}], [{'rank': '6', 'suit': 'spades'}, {'rank': 'K', 'suit': 'spades'}]], 'blind': 4, 'initial_stacks': [994, 986, 22, 988, 2010], 'community_cards': [{'r

([[array([[[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  1.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  1.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]],
   
          [[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.]],
   
          [[ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.],


In [4]:
inputs = []
outputs = []

for file in glob.glob("/data/poker/rawdata/*.json"):
    f = open(file, 'r')
    hands_in_game = json.loads(f.read())

    for hand in hands_in_game: 
        inp, outp = convert_hand(hand)
        inputs.extend(inp)
        outputs.extend(outp)

inputs = np.array(inputs)
outputs = np.array(outputs)

In [5]:
inputs.shape

(70338, 2)

In [6]:
outputs.shape

(70338, 1)

In [7]:
import bcolz

ifile = bcolz.carray(inputs, rootdir='/data/poker/regress_on_bets_input.dat', mode='w')
ifile.flush()
ofile = bcolz.carray(outputs, rootdir='/data/poker/regress_on_bets_output.dat', mode='w')
ofile.flush()